# Day 20 - Reassembling images

* https://adventofcode.com/2020/day/20

I've treated this as a path finding problem; the search state only needs to track what the remaining top and left edge values of the unplaced tiles.

Of course, you need to be able to determine what the edge values are; there are 8 different ways an image can be oriented through rotation or flipping, and we can know, up front, what part of the input data we need to use for this.

In [1]:
import operator
from collections import deque
from collections.abc import Iterator, MutableSet, Sequence, Set
from dataclasses import dataclass, field
from enum import Enum
from functools import cached_property, lru_cache, partial, reduce
from itertools import product
from typing import cast, Callable, Dict, Generic, List, Mapping, Optional, Tuple, TypeVar

import numpy as np


class Edge(Enum):
    top = 0
    right = 1
    bottom = 2
    left = 3


class EdgeDir(Enum):
    tf = (0, slice(None, None, 1))
    tr = (0, slice(None, None, -1))
    rf = (slice(None, None, 1), -1)
    rr = (slice(None, None, -1), -1)
    bf = (-1, slice(None, None, 1))
    br = (-1, slice(None, None, -1))
    lf = (slice(None, None, 1), 0)
    lr = (slice(None, None, -1), 0)


class Orientation(Enum):
    orig = (EdgeDir.tf, EdgeDir.rf, EdgeDir.bf, EdgeDir.lf, False, 0)
    rot90 = (EdgeDir.lr, EdgeDir.tf, EdgeDir.rr, EdgeDir.bf, False, 1)
    rot180 = (EdgeDir.br, EdgeDir.lr, EdgeDir.tr, EdgeDir.rr, False, 2)
    rot270 = (EdgeDir.rf, EdgeDir.br, EdgeDir.lf, EdgeDir.tr, False, 3)
    mirror = (EdgeDir.tr, EdgeDir.lf, EdgeDir.br, EdgeDir.rf, True, 0)
    mrt90 = (EdgeDir.rr, EdgeDir.tr, EdgeDir.lr, EdgeDir.br, True, 1)
    mrt180 = (EdgeDir.bf, EdgeDir.rr, EdgeDir.tf, EdgeDir.lr, True, 2)
    mrt270 = (EdgeDir.lf, EdgeDir.bf, EdgeDir.rf, EdgeDir.tf, True, 3)

    def transpose(self, m: "np.array[bool]") -> "np.array[bool]":
        if self.value[-2]:
            m = np.fliplr(m)
        if rotations := self.value[-1]:
            m = np.rot90(m, 4 - rotations)  # rotations counter-clockwise
        return m


@dataclass(frozen=True)
class Tile:
    id: int
    data: str

    @cached_property
    def matrix(self) -> "np.array[np.bool]":
        return np.array(
            [c == "#" for line in self.data.splitlines() for c in line]
        ).reshape((-1, self.data.index("\n")))

    @classmethod
    def from_data(cls, data: str) -> "Tile":
        tile_line, data = data.split("\n", 1)
        tile_id = int(tile_line.split()[1].strip(":"))
        return cls(tile_id, data)

    def __str__(self):
        return f"Tile {self.id}:\n{self.data}"

    @cached_property
    def orientations(self) -> "Set[OrientedTile]":
        return frozenset(
            OrientedTile(self, o)
            for o in Orientation
        )

    @lru_cache(maxsize=None)
    def __getitem__(self, edgedir: EdgeDir) -> int:
        return (
            np.packbits(np.pad(self.matrix[edgedir.value], (6, 0))).view(">u2").item()
        )


@dataclass(frozen=True)
class OrientedTile:
    tile: Tile
    orientation: Orientation

    @property
    def id(self) -> int:
        return self.tile.id

    def __str__(self) -> str:
        formatted = "\n".join(
            ["".join([".#"[int(v)] for v in row]) for row in self.matrix]
        )
        return f"Oriented tile {self.id}, {self.orientation.name}\n{formatted}"

    @lru_cache(maxsize=None)
    def __getitem__(self, edge: Edge) -> int:
        return self.tile[self.orientation.value[edge.value]]

    @cached_property
    def matrix(self) -> "np.array[np.bool]":
        return self.orientation.transpose(self.tile.matrix)


@dataclass(frozen=True)
class SearchState:
    size: int = 0
    diag: int = 0
    x: int = 0
    y: int = 0
    state: Sequence[Optional[OrientedTile]] = ()
    tops: Mapping[int, Set[OrientedTile]] = field(hash=False, default_factory=dict)
    lefts: Mapping[int, Set[OrientedTile]] = field(hash=False, default_factory=dict)

    @classmethod
    def from_data(cls, data: str) -> "SearchState":
        tile_set = frozenset(Tile.from_data(chunk) for chunk in data.split("\n\n"))
        size = int(len(tile_set) ** 0.5)  # square root of the initial set
        tops: Dict[int, MutableSet[OrientedTile]] = {}
        lefts: Dict[int, MutableSet[OrientedTile]] = {}
        for tile in tile_set:
            for orientated in tile.orientations:
                tops.setdefault(orientated[Edge.top], set()).add(orientated)
                lefts.setdefault(orientated[Edge.left], set()).add(orientated)
        return cls(size, tops=tops, lefts=lefts)

    @property
    def complete(self) -> bool:
        return bool(self.size) and self.size - 1 == self.x == self.y

    @property
    def checksum(self) -> int:
        if not self.complete:
            raise ValueError("Incomplete state")
        state, size = self.state, self.size
        corners = (state[x + size * y] for x, y in product((0, size - 1), repeat=2))
        return reduce(operator.mul, (s.id for s in corners if s is not None))
    
    @property
    def image(self) -> "np.array[np.bool]":
        if not self.complete:
            raise ValueError("Incomplete state")
        state, size = cast(Sequence[OrientedTile], self.state), self.size
        rows = [
            np.concatenate([state[x + (size * y)].matrix[1:-1, 1:-1] for x in range(size)], axis=1)
            for y in range(size)
        ]
        return np.concatenate(rows, axis=0)

    def __str__(self):
        state, size = self.state, self.size
        as_matrix = [[state[x + y * size] for x in range(size)] for y in range(size)]
        formatted = "\n".join(
            [
                "    ".join(["None" if s is None else format(s.id, ">4d") for s in r])
                for r in as_matrix
            ]
        )
        return f"Filled: {1 - self.to_place:.2%}\n{formatted}"

    def _starting_states(self) -> Iterator["SearchState"]:
        tops, lefts = self.tops, self.lefts
        state_tail = (None,) * (self.size ** 2 - 1)
        allvariants = set.union(*tops.values())  # type: ignore  # Yes, these are sets.
        for ort in sorted(allvariants, key=lambda ot: (-ot.id, ot.orientation.name)):
            yield SearchState(
                self.size,
                state=(ort, *state_tail),
                tops={v: os - ort.tile.orientations for v, os in tops.items()},
                lefts={v: os - ort.tile.orientations for v, os in lefts.items()},
            )

    def neighbors(self) -> Iterator["SearchState"]:
        if not (state := self.state):
            yield from self._starting_states()
            return

        size = self.size
        diag, x, y = self.diag, self.x, self.y

        # along the diagonal from top left to bottom right, produce
        # the full line to the right, and the column to the bottom
        if y == diag and x < size:
            x += 1
            if x == size:
                x = diag
                y += 1
        elif y < size:
            y += 1
            if y == size:
                diag += 1
                x = y = diag

        matched = None
        tops, lefts = self.tops, self.lefts
        if y and (below := state[x + size * (y - 1)]):
            matched = tops.get(below[Edge.bottom], None)
        if x and (next_to := state[x - 1 + size * y]):
            matched_left = lefts.get(next_to[Edge.right], frozenset())
            if matched is None:
                matched = matched_left
            else:
                matched &= matched_left
        if not matched:
            return

        state_lead, state_tail = state[: x + size * y], state[x + size * y + 1 :]
        new_searchstate = partial(SearchState, size=size, diag=diag, x=x, y=y)
        for ort in sorted(matched, key=lambda ot: (-ot.id, ot.orientation.name)):
            yield new_searchstate(
                state=(*state_lead, ort, *state_tail),
                tops={v: os - ort.tile.orientations for v, os in tops.items()},
                lefts={v: os - ort.tile.orientations for v, os in lefts.items()},
            )


def reconstruct_image(
    tiledata: str,
    anim_callback: Optional[Callable[[SearchState], None]] = None
) -> SearchState:
    start = SearchState.from_data(tiledata)
    stack = deque([start])

    while stack:
        current = stack.pop()
        if anim_callback:
            anim_callback(current)
        if current.complete:
            return current

        stack.extend(current.neighbors())

    raise AssertionError("No solution found")


testdata = (
    "Tile 2311:\n..##.#..#.\n##..#.....\n#...##..#.\n####.#...#\n"
    "##.##.###.\n##...#.###\n.#.#.#..##\n..#....#..\n###...#.#.\n..###..###\n\n"
    "Tile 1951:\n#.##...##.\n#.####...#\n.....#..##\n#...######\n"
    ".##.#....#\n.###.#####\n###.##.##.\n.###....#.\n..#.#..#.#\n#...##.#..\n\n"
    "Tile 1171:\n####...##.\n#..##.#..#\n##.#..#.#.\n.###.####.\n"
    "..###.####\n.##....##.\n.#...####.\n#.##.####.\n####..#...\n.....##...\n\n"
    "Tile 1427:\n###.##.#..\n.#..#.##..\n.#.##.#..#\n#.#.#.##.#\n"
    "....#...##\n...##..##.\n...#.#####\n.#.####.#.\n..#..###.#\n..##.#..#.\n\n"
    "Tile 1489:\n##.#.#....\n..##...#..\n.##..##...\n..#...#...\n"
    "#####...#.\n#..#.#.#.#\n...#.#.#..\n##.#...##.\n..##.##.##\n###.##.#..\n\n"
    "Tile 2473:\n#....####.\n#..#.##...\n#.##..#...\n######.#.#\n"
    ".#...#.#.#\n.#########\n.###.#..#.\n########.#\n##...##.#.\n..###.#.#.\n\n"
    "Tile 2971:\n..#.#....#\n#...###...\n#.#.###...\n##.##..#..\n"
    ".#####..##\n.#..####.#\n#..#.#..#.\n..####.###\n..#.#.###.\n...#.#.#.#\n\n"
    "Tile 2729:\n...#.#.#.#\n####.#....\n..#.#.....\n....#..#.#\n"
    ".##..##.#.\n.#.####...\n####.#.#..\n##.####...\n##..#.##..\n#.##...##.\n\n"
    "Tile 3079:\n#.#.#####.\n.#..######\n..#.......\n######....\n"
    "####.#..#.\n.#...#.##.\n#.#####.##\n..#.###...\n..#.......\n..#.###..."
)

test_state = reconstruct_image(testdata)
assert test_state.checksum == 20899048083289

In [2]:
import aocd
data = aocd.get_data(day=20, year=2020)

In [3]:
image_state = reconstruct_image(data)
print("Part 1:", image_state.checksum)

Part 1: 11788777383197


## Part 2 - finding the sea monsters

Once again the SciPy toolkit makes this very easy: by using [`scipy.ndimage.correlate()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.correlate.html) we get a count, at every position in an image, how many points in a kernel (our seamonster) are present around that point. If that number matches the number of pixels in the monster image, we have found a sea monster!

In [4]:
from scipy.ndimage import correlate

seamonster = """\
..................#.
#....##....##....###
.#..#..#..#..#..#...
"""


def assess_roughness(image: "np.array[bool]") -> int:
    sm_kernel = (
        np.array([c == "#" for line in seamonster.splitlines() for c in line])
        .reshape((-1, seamonster.index("\n")))
        .astype(np.uint8)
    )
    monster_size = sm_kernel.sum()
    transposed_images = [
        orientation.transpose(image).astype(np.uint8) for orientation in Orientation
    ]
    return next(
        int(img.sum() - (mcount * monster_size))
        for img in transposed_images
        if (
            mcount := (correlate(img, sm_kernel, mode="constant") == monster_size).sum()
        )
    )


assert assess_roughness(test_state.image) == 273

In [5]:
print("Part 2:", assess_roughness(image_state.image))

Part 2: 2242


I created a visualisation of the image reconstruction process; when reading this online you may want to use [the Jupyter notebook viewer](https://nbviewer.jupyter.org/github/mjpieters/adventofcode/blob/master/2020/Day%2020.ipynb); the GitHub renderer filters them out.

In [6]:
%matplotlib inline

from itertools import product

import easing_functions
import matplotlib.pyplot as plt
from matplotlib import animation
from PIL import Image, ImageChops, ImageDraw

plt.rc('animation', html='html5')

def tile_image(otile: OrientedTile) -> Image:
    colour = (otile.id & 0xF00) >> 4, (otile.id & 0xF0), (otile.id & 0xF) << 4
    width, height = otile.matrix.shape
    base = Image.new("RGB", (width, height), colour)
    draw = ImageDraw.Draw(base)
    from_ = (0, 0)
    for edge, to in zip(
        Edge, ((width - 1, 0), (width - 1, height - 1), (0, height - 1), (0, 0))
    ):
        edge_id = otile[edge]
        # 10 bits, divided up into 4, 3 and 3 bits, to make light RGB colours
        colour = (edge_id & 0x3C0) >> 2, (edge_id & 0x38) << 2, (edge_id & 7) << 5
        draw.line((from_, to), colour, 1)
        from_ = to
    return ImageChops.multiply(base, Image.fromarray(~otile.matrix).convert("RGB"))

def animate(tiledata: str, scale: int = 1) -> Sequence[Image]:
    frames = []
    frame_cache = {}
    tiles: Dict[OrientedTile, Image] = {}

    # animation properties
    fps = 20  # frames per second
    bg = (0xFF, 0xFF, 0xFF)
    snake_colour = (0x3F, 0x7F, 0xFF)

    pause_before_merge = fps // 4
    merge_duration = fps // 1
    rot_duration = fps // 4
    scan_speed = 3  # frames per tiles line
    hold_final_frame = fps // 1

    def init_frames(state: SearchState):
        tiles.update({
            ort: tile_image(ort) for ort in set.union(*state.tops.values())  # type: ignore  # Yes, these are sets.
        })
        tw, th = next(iter(tiles.values())).size
        shape = (state.size * (tw + 1) + 1, state.size * (th + 1) + 1)

        frame_cache[(None,) * (state.size ** 2)] = frame = Image.new("RGB", shape, bg)
        frames.append(frame)

    def process_state(state: SearchState):
        if not state.state:
            init_frames(state)
            return

        otile = state.state[state.x + state.y * state.size]
        tileimg = tiles[otile]
        frame_cache_key = tuple(s if s is not otile else None for s in state.state)
        frame_cache[state.state] = frame = frame_cache[frame_cache_key].copy()
        frame.paste(
            tileimg,
            ((tileimg.width + 1) * state.x + 1, (tileimg.height + 1) * state.y + 1),
        )
        frames.append(frame)

    def merge_tiles(state: SearchState):
        # repeat final frame
        frames.extend(pause_before_merge * [frames[-1]])

        offset = (state.size * 3) // 2
        tw = state.state[0].matrix.shape[0]
        easing = easing_functions.QuadEaseInOut
        # paths describe the animation path for x or y, indexed by those coordinates
        paths = [
            easing(start=(tw + 1) * x + 1, end=(tw - 2) * x + offset, duration=merge_duration)
            for x in range(state.size)
        ]
        alpha = easing(start=0xFF, end=0, duration=merge_duration)
        base = Image.new("RGB", frames[-1].size, bg)
        for f in range(merge_duration):
            frame = base.copy()
            for i, otile in enumerate(state.state):
                tx, ty = i % state.size, i // state.size
                tileimg = tiles[otile]
                # mask out the edge and background pixels, as alpha channel
                # fading out.
                mask = otile.matrix.astype(np.uint8) * 255
                mask[(0, -1), :], mask[:, (0, -1)] = 0, 0
                mask[mask == 0] = round(alpha.ease(f))

                frame.paste(
                    tileimg,
                    (round(paths[tx].ease(f)), round(paths[ty].ease(f))),
                    Image.fromarray(mask),
                )
            frames.append(frame)
    
    def scan_for_snakes(state: SearchState):
        base = Image.new("RGB", frames[-1].size, bg)
        sat_image = Image.fromarray(~state.image).convert("RGB")
        tsize = state.state[0].matrix.shape[0] - 2
        size = state.size
        offset = (size * 3) // 2 + 1
        duration = (state.size + 2) * scan_speed

        # build full-colour tile image for the scanner beam
        scan_overlay = Image.new("RGB", (size * tsize,) * 2, bg)
        for i, otile in enumerate(state.state):
            tx, ty = i % size, i // size
            tileimg = tiles[otile].crop((1, 1, 1 + tsize, 1 + tsize))
            scan_overlay.paste(tileimg, (tsize * tx, tsize * ty))

        # scanning mask, to be used as a mask selecting what to copy from
        # the scan overlay. The scan mask is animated to pass from top to
        # bottom.
        scan_m = np.zeros(base.size, dtype=np.uint8)
        scan_m[:7, :] = 100
        scan_m[1:6, :] += 100
        scan_m[2:5, :] += 55
        scan_band = Image.fromarray(scan_m)
        scan_path = easing_functions.QuadEaseInOut(
            start=offset - 6, end=offset + (size * tsize), duration=duration
        )

        # sea monster kernel and image (solid colour rectangle and alpha mask)
        sm_kernel = (
            np.array([c == "#" for line in seamonster.splitlines() for c in line])
            .reshape((-1, seamonster.index("\n")))
            .astype(np.uint8)
        )
        monster_size = sm_kernel.sum()
        sm_rect = Image.new("RGB", sm_kernel.shape[::-1], snake_colour)
        sm_mask = Image.fromarray(sm_kernel * 255)

        previous_orientation = None
        for orientation in Orientation:
            orientated = orientation.transpose(state.image)
            image = Image.fromarray(~orientated).convert("RGB")

            if previous_orientation is not None:
                # animate moving between orientations
                rotation = orientation.value[-1]
                if rotation:
                    # all rotations are +90 from previous state
                    rot_easing = easing_functions.QuadEaseInOut(start=0, end=-90, duration=rot_duration)
                    cw, ch = offset + previous_orientation.shape[1] // 2, offset + previous_orientation.shape[0] // 2
                    to_rotate = Image.fromarray(~previous_orientation).convert("RGB")
                    for f in range(rot_duration):
                        rotated = to_rotate.rotate(rot_easing.ease(f), Image.BICUBIC, expand=True, fillcolor=bg)
                        frame = base.copy()
                        frame.paste(rotated, (cw - rotated.width // 2, ch - rotated.height // 2 ))
                        frames.append(frame)
                else:  # flipped
                    tsize = previous_orientation.shape[0]
                    flip_easing = easing_functions.QuadEaseInOut(start=-tsize, end=tsize, duration=rot_duration)
                    prev_image = Image.fromarray(~previous_orientation).convert("RGB")
                    for f in range(rot_duration):
                        frame = base.copy()
                        eased = round(flip_easing.ease(f))
                        to_squeeze = prev_image if eased < 0 else image
                        squeezed_size = abs(eased)
                        squeezed = to_squeeze.resize((squeezed_size, squeezed_size))
                        toffset = offset + (tsize - squeezed_size) // 2
                        frame.paste(squeezed, (toffset, toffset))
                        frames.append(frame)

            previous_orientation = orientated
            correlation = correlate(
                orientated.astype(np.uint8), sm_kernel, mode="constant"
            )
            snake_positions = [
                (offset + x - sm_rect.width // 2, offset + y - sm_rect.height // 2)
                for y, x in zip(*np.where(correlation == monster_size))
            ]

            frame_base = base.copy()
            frame_base.paste(image, (offset, offset))
            scanner_beam = base.copy()
            scanner_beam.paste(
                Image.fromarray(orientation.transpose(np.array(scan_overlay))),
                (offset, offset),
            )

            for f in range(duration):
                frame = frame_base.copy()
                scanner_y = int(scan_path.ease(f))
                scan_mask = ImageChops.offset(scan_band, 0, scanner_y)
                for sx, sy in snake_positions:
                    if sy <= scanner_y:
                        frame.paste(sm_rect, (sx, sy), sm_mask)
                frame.paste(scanner_beam, (0, 0), scan_mask)
                frames.append(frame)
            
            if snake_positions:
                easing = easing_functions.QuadEaseInOut(start=255, end=0, duration=hold_final_frame // 2)
                for f in range(hold_final_frame):
                    frame = base.copy()
                    if f < hold_final_frame // 2:
                        frame.paste(frame_base, (0, 0), Image.new("L", frame_base.size, round(easing.ease(f))))
                    for sx, sy in snake_positions:
                        frame.paste(sm_rect, (sx, sy), sm_mask)
                    frames.append(frame)
                return                

    fig, ax = plt.subplots(figsize=(12, 12))
    fig.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=0, hspace=0)
    ax.set_axis_off()

    final_state = reconstruct_image(tiledata, process_state)
    del frame_cache
    merge_tiles(final_state)
    scan_for_snakes(final_state)

    scaled = frames[0].width * scale, frames[0].height * scale
    imgs = [[plt.imshow(f.resize(scaled, Image.NEAREST), animated=True)] for f in frames]
    anim = animation.ArtistAnimation(fig, imgs, interval=1000 // fps, blit=True, repeat=False)

    plt.close(fig)

    return anim

animate(data, scale=15)